In [1]:
import os
import re
from pathlib import Path
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [2]:
# Parameter
input_dir = Path("data/processed")
chunk_size = 200
chunk_overlap = 0
index_path = "faiss_index"
model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [3]:
# Text splitter konfigurieren
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", "."]
)

In [4]:
# Alle Dokumente sammeln
all_documents = []

for md_file in input_dir.glob("*.md"):

    # Dateiname analysieren
    filename = md_file.stem  # z. B. "bmw_2023_filtered"
    parts = filename.split("_")  # → ["bmw", "2023", "filtered"]

    # Metadaten extrahieren
    company = parts[0]
    year = parts[1]

    with md_file.open("r", encoding="utf-8") as f:
        text = f.read()

    text = text.replace('\u202f', ' ') # Filter 1: Unicode entfernen
    text = re.sub(r'(?<=\d)\.(?=\d)', '', text) # Filter 2: Entferne Punkte zwischen zwei Ziffern (z. B. 1.234 → 1234)
    text = re.sub(r'(?<=\w)-\s(?=\w)', '', text) # Filter 3: Entferne Trennung mit - Leerzeichen (Kon- zern → Konzern)

    chunks = text_splitter.split_text(text)
    
    for i, chunk in enumerate(chunks):
        all_documents.append(Document(
            page_content=chunk,
            metadata={"company": company, "year": year, "chunk": i}
        ))

In [5]:
bmw_chunks = [doc for doc in all_documents if doc.metadata["company"].lower() == "bmw"]
bmw_chunks[:20]

[Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 0}, page_content='Auf dem Weg zur elektrischen und digitalisierten Mobilität in einer nachhaltigen Kreislaufwirtschaft'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 1}, page_content='. Bericht über die ökonomische Leistung der BMW Group und ihren ökologischen und gesellschaftlichen Beitrag.'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 2}, page_content='BMW Group Bericht 2023 An unsere Stakeholder Zusammengefasster Lagebericht Konzernabschluss Bilanzeid und Vermerke des Prüfers Vergütungsbericht Weitere Informationen'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 3}, page_content='Die BMW Group blickt auf ein erfolgreiches Jahr 2023. Das Unternehmen wächst und ist profitabel zugleich. Die Marke BMW ist erneut die Nummer eins im Premiumsegment'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 4}, page_content='. Über alle Marken, Segmente und Antriebs

In [6]:
# Embeddings vorbereiten
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/var/folders/9q/9wk8_fhx2zgbpf5xybsmn49w0000gn/T/ipykernel_32363/2710609826.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/Users/yanoothan/Master/Advanced NLP/BWLLM/BWLLM_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# FAISS-Index erstellen
faiss_index = FAISS.from_documents(all_documents, embedding=embeddings)

In [8]:
# Lokal speichern
faiss_index.save_local(index_path)